# Data Cleaning and Feature Engineering

## 1. Import Packages

In [1]:
import boto3
import pandas as pd
import datetime
import numpy as np
import awswrangler as wr
from ydata_profiling import ProfileReport

/Users/zori/Documents/ML-Project/AWS-pipeline/nutrisage-mlops/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sess = boto3.session

## 2. Import Data from S3 Bucket

In [3]:
s3_bucket = "s3://nutrisage/data"
data = pd.read_parquet(s3_bucket)

In [4]:
df = data.sample(frac=0.1, random_state=42)
df.reset_index(inplace=True, drop=True)
df.head()

,energy_100g,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,product_name,main_category,categories_tags,brands_tags,countries_tags,serving_size,created_t,nutrition_grade_fr,year,country
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Beef Stew Seasoning Mix,<NA>,[en:undefined],[iga],[en:united-states],4g,1489067206,unknown,2017,united-states
1,607.0,145.0,2.7,2.7,0.7,0.6,NaN,12.3,0.00,Oeufs frais de Martinique,<NA>,"[en:farming-products, en:eggs, en:chicken-eggs...",[xx:leader-price],[en:france],<NA>,1539195077,a,2018,france
2,515.0,123.0,3.1,1.4,0.5,0.5,NaN,23.0,0.80,Le jambon de chez nous,<NA>,[],[],[en:france],<NA>,1571401079,unknown,2019,france
3,1824.0,436.0,27.0,18.0,45.0,36.0,NaN,2.8,1.32,buchette chocolat,<NA>,[],[],[en:france],<NA>,1576867782,unknown,2019,france
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dumėnil,<NA>,[],[],[en:france],<NA>,1543916596,unknown,2018,france


In [5]:
df.to_parquet("../data/raw/sample_data.parquet")

In [6]:
print(df.shape)
df.isna().sum()

(98489, 19)


energy_100g           16002
energy-kcal_100g      18235
fat_100g              16469
saturated-fat_100g    18994
carbohydrates_100g    16488
sugars_100g           17752
fiber_100g            66120
proteins_100g         16381
sodium_100g           17949
product_name           1911
main_category         98489
categories_tags           0
brands_tags               0
countries_tags            0
serving_size          69994
created_t                 0
nutrition_grade_fr        7
year                      0
country                   0
dtype: int64

In [7]:
df.describe()

,energy_100g,energy-kcal_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,sodium_100g,created_t
count,82487.000000,80254.000000,82020.000000,79495.000000,82001.000000,80737.000000,32369.000000,82108.000000,80540.000000,98489.0
mean,1257.250000,301.949402,15.277145,5.660735,31.454763,15.432584,3.485418,9.099054,0.550386,1533651160.216593
std,1368.245728,330.161072,27.325125,10.887858,45.901936,32.038227,7.034051,14.639171,4.708497,31178065.910814
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1483291953.0
25%,435.000000,105.000000,0.700000,0.100000,3.560000,0.700000,0.000000,1.300000,0.012000,1506544222.0
50%,1125.000000,271.000000,7.142879,1.900000,15.800000,3.970000,1.600000,6.000000,0.172000,1536445887.0
75%,1720.000000,412.000000,22.000000,7.700000,54.400002,20.000000,4.000000,12.500000,0.520000,1561480332.0
max,66944.000000,16000.000000,1220.000000,714.000000,1670.000000,1670.000000,276.000000,1430.000000,870.857300,1609428462.0


In [8]:
drop_cols = ['energy_100g', 'saturated-fat_100g', 'fiber_100g', 'main_category', \
             'categories_tags', 'brands_tags', 'countries_tags', 'serving_size', \
            'created_t', 'year', 'country']

In [9]:
df_clean = df.drop(columns=drop_cols)
df_clean.head()

,energy-kcal_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,sodium_100g,product_name,nutrition_grade_fr
0,NaN,NaN,NaN,NaN,NaN,NaN,Beef Stew Seasoning Mix,unknown
1,145.0,2.7,0.7,0.6,12.3,0.00,Oeufs frais de Martinique,a
2,123.0,3.1,0.5,0.5,23.0,0.80,Le jambon de chez nous,unknown
3,436.0,27.0,45.0,36.0,2.8,1.32,buchette chocolat,unknown
4,NaN,NaN,NaN,NaN,NaN,NaN,Dumėnil,unknown


In [10]:
df_clean['nutrition_grade_fr'].value_counts()

nutrition_grade_fr
unknown           42106
e                 15817
d                 13789
c                 10450
a                  7384
b                  5752
not-applicable     3184
Name: count, dtype: Int64

In [11]:
df_clean = df_clean.loc[~df['nutrition_grade_fr'].isin(('unknown', 'not-applicable')), :]
df_clean.head()

,energy-kcal_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,sodium_100g,product_name,nutrition_grade_fr
1,145.0,2.700000,0.700000,0.6,12.30,0.000,Oeufs frais de Martinique,a
5,542.0,33.330002,58.330002,37.5,4.17,0.000,"Market pantry, covered cookies with espresso c...",e
7,194.0,7.500000,27.799999,24.6,3.40,0.040,Sweet lovin' strawberry with vanilla meringue,d
9,1.0,0.000000,0.300000,0.0,0.20,0.000,Delicious Café fe Colombia,b
11,48.0,0.100000,11.000000,9.0,0.60,0.004,Zumo de naranja,c


In [12]:
df_clean['nutrition_grade_fr'].value_counts()

nutrition_grade_fr
e    15817
d    13789
c    10450
a     7384
b     5752
Name: count, dtype: Int64

In [13]:
# remove outliers
for col in df_clean.columns[1:6]:
    df_clean = df_clean.loc[~(df_clean[col] > 100), :]

In [14]:
df_clean.head()

,energy-kcal_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,sodium_100g,product_name,nutrition_grade_fr
1,145.0,2.700000,0.700000,0.6,12.30,0.000,Oeufs frais de Martinique,a
5,542.0,33.330002,58.330002,37.5,4.17,0.000,"Market pantry, covered cookies with espresso c...",e
7,194.0,7.500000,27.799999,24.6,3.40,0.040,Sweet lovin' strawberry with vanilla meringue,d
9,1.0,0.000000,0.300000,0.0,0.20,0.000,Delicious Café fe Colombia,b
11,48.0,0.100000,11.000000,9.0,0.60,0.004,Zumo de naranja,c


In [15]:
df_clean.describe()

,energy-kcal_100g,fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,sodium_100g
count,49443.000000,51029.000000,51008.000000,50570.000000,51029.000000,51052.000000
mean,277.078125,14.634240,27.252441,13.147551,8.372143,0.453823
std,196.395950,18.833599,27.718578,19.264891,8.837975,1.643786
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,105.000000,0.800000,3.300000,0.667750,1.400000,0.016000
50%,261.000000,7.500000,14.000000,3.600000,6.000000,0.200000
75%,402.000000,23.000000,52.000000,17.860001,12.200000,0.520000
max,1799.000000,100.000000,100.000000,100.000000,92.000000,100.000000


In [16]:
df_clean['nutrition_grade_fr'].value_counts()

nutrition_grade_fr
e    14798
d    13547
c    10335
a     7292
b     5738
Name: count, dtype: Int64

In [22]:
df_clean.shape

(51717, 8)

In [26]:
df_clean = df_clean.loc[~df_clean['nutrition_grade_fr'].isna(), :]

In [27]:
df_clean['nutrition_grade_fr'].isna().sum()

np.int64(0)

In [29]:
df_clean.shape

(51710, 8)

In [30]:
df_clean.to_parquet("../data/processed/clean_data.parquet")